In [216]:
%%writefile streamlit_app.py

import streamlit as st
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

model = joblib.load('xgb_model.pkl')

def predict_price(input_data):
    input_df = pd.DataFrame([input_data])
    prediction = model.predict(input_df)
    return prediction

def main():
    st.markdown("""
        <style>
        .stApp {
            background-image: url("https://images.pexels.com/photos/258154/pexels-photo-258154.jpeg?cs=srgb&dl=pexels-pixabay-258154.jpg&fm=jpg");
            background-size: cover;
            background-repeat: no-repeat;
            background-attachment: fixed;
        }
        h1 {
            color: #8B0000;
        }
        .stSelectbox .css-2b097c-container, .stNumberInput .css-1d3z3hw {
            background-color: #FFFFFF; 
            border-radius: 10px;
            border: 2px solid #8B0000; 
        }
        .stSelectbox .css-1uccc91-singleValue, .stNumberInput input {
            color: #FFFFFF; 
        }
        </style>
        """, unsafe_allow_html=True)
    
    
    if 'prediction_history' not in st.session_state:
        st.session_state['prediction_history'] = pd.DataFrame(columns=['Host Total Listings Count', 'Room Type', 'Accommodates', 'Bathrooms', 'Bedrooms','Beds','Bed Type','Cleaning Fee','Extra People','Minimum Nights',
                                                                       'Number of Reviews','Review Scores Cleanliness','Review Scores Checkin','Review Scores Communication','Review Scores Location','Review Rating',
                                                                       'Host Response Rate','Category','Cancellation Type','Amenities Score','Price Bins', 'Predicted Price'])
    
    st.title('Airbnb Price Prediction')
    
    category_mapping = {
        0: 'Apartments', 1: 'Traditional', 2: 'Luxe', 3: 'Creative Spaces', 4: 'Exotic'
    }
    bed_type_mapping = {
        0: 'Real Bed', 1: 'Futon', 2: 'Airbed', 3: 'Pull-out Sofa', 4: 'Couch'
    }
    room_type_mapping = {
        0: 'Entire home/apt', 1: 'Private room', 2: 'Shared room'
    }
    cancellation_type_mapping = {
        0: 'Strict', 1: 'Flexible', 2: 'Moderate'
    }

    left, right = st.columns(2)

    with left:
        room_type = st.selectbox('Room Type', options=list(room_type_mapping.keys()), format_func=lambda x: room_type_mapping[x])
        bed_type = st.selectbox('Bed Type', options=list(bed_type_mapping.keys()), format_func=lambda x: bed_type_mapping[x])
        category = st.selectbox('Category', options=list(category_mapping.keys()), format_func=lambda x: category_mapping[x])
        cancellation_type = st.selectbox('Cancellation Type', options=list(cancellation_type_mapping.keys()), format_func=lambda x: cancellation_type_mapping[x])
        accommodates = st.number_input('Accommodates', min_value=1)
        bathrooms = st.number_input('Bathrooms', min_value=1)

    with right:
        bedrooms = st.number_input('Bedrooms', min_value=1)
        beds = st.number_input('Beds', min_value=1)
        cleaning_fee = st.number_input('Cleaning Fee', min_value=0)
        extra_people = st.number_input('Extra People', min_value=0)
        minimum_nights = st.number_input('Minimum Nights', min_value=1)

    if st.button('Predict Price'):
        input_data = {
            'Host Total Listings Count': np.random.randint(1, 1115),
            'Room Type': room_type,
            'Accommodates': accommodates,
            'Bathrooms': bathrooms,
            'Bedrooms': bedrooms,
            'Beds': beds,
            'Bed Type': bed_type,
            'Cleaning Fee': cleaning_fee,
            'Extra People': extra_people,
            'Minimum Nights': minimum_nights,
            'Number of Reviews': np.random.randint(0, 750),
            'Review Scores Cleanliness': np.random.randint(1, 10),
            'Review Scores Checkin': np.random.randint(1, 10),
            'Review Scores Communication': np.random.randint(1, 10),
            'Review Scores Location': np.random.randint(1, 10),
            'Review Rating': np.random.randint(0, 100),
            'Host Response Rate': np.random.randint(0, 100),
            'Category': category,
            'Cancellation Type': cancellation_type,
            'Amenities Score': np.random.uniform(0, 10),
            'Price Bins': np.random.randint(0, 9)        
        }
        
        prediction = predict_price(input_data)
        input_data['Predicted Price'] = prediction[0]
        new_row = pd.DataFrame([input_data])
        st.session_state['prediction_history'] = pd.concat([st.session_state['prediction_history'], new_row], ignore_index=True)
        st.success(f'Estimated Airbnb Price: ${prediction[0]:,.2f}')
    
        if not st.session_state['prediction_history'].empty:
            fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(15, 12))
            plot_mappings = [category_mapping, bed_type_mapping, room_type_mapping, cancellation_type_mapping]
            titles = ['Category', 'Bed Type', 'Room Type', 'Cancellation Type']
            for mapping, title, ax in zip(plot_mappings, titles, axes.flatten()):
                plot_data = st.session_state['prediction_history'].copy()
                plot_data[title] = plot_data[title].map(mapping)
                sns.barplot(x=title, y='Predicted Price', data=plot_data, estimator=np.mean, ax=ax)
                ax.set_title(f'Predicted Price vs.  {title}')
                ax.set_xlabel(title)
                ax.set_ylabel('Predicted Price($)')
                ax.tick_params(axis='x', rotation=45)
            plt.tight_layout()
            st.pyplot(fig)
            
            features = ['Accommodates', 'Bathrooms', 'Bedrooms', 'Beds', 'Cleaning Fee', 'Extra People']
            fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(15, 20))
            for feature, ax in zip(features, axes.flatten()):
                sns.lineplot(x=feature, y='Predicted Price', data=st.session_state['prediction_history'], ax=ax)
                ax.set_title(f'Predicted Price vs. {feature}')
                ax.set_xlabel(feature)
                ax.set_ylabel('Predicted Price($)')
            plt.tight_layout()
            st.pyplot(fig)
            
        
if __name__ == '__main__':
    main()

Overwriting streamlit_app.py


In [ ]:
!streamlit run streamlit_app.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://10.84.59.128:8501

/Users/arvindprashanth/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [20:48:55] WARNING: /Users/runner/work/xgboost/xgboost/src/gbm/../common/error_msg.h:80: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  warnings.warn(smsg, UserWarning)
[20:49:13] WARNING: /Users/runner/work/xgboost/xgboost/src/gbm/../common/error_msg.h:80: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model